<a href="https://colab.research.google.com/github/Jinhan0603/OpenBot/blob/master/policy/policy_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 셀 A 최초 1회
**목적** Opnebot과 Colab 환경 맞추기
> Colab(Py3.12)과 맞는 TF/Numpy 조합 설치



In [1]:
# [한글] Colab(Py3.12)과 호환되는 기본 스택 설치
# - TF 2.20.0 + NumPy 2.0.x 조합
# - ml-dtypes는 TF가 자동으로(>=0.5.1) 설치하므로 지정하지 않습니다.

!pip uninstall -y jax jaxlib chex optax flax \
  tensorflow-text tensorflow-decision-forests tf-keras tensorstore \
  orbax-checkpoint tensorflow-hub dopamine-rl albumentations albucore \
  pymc pytensor bigframes ydf -qq

!pip install -q --upgrade --force-reinstall tensorflow==2.20.0 numpy==2.0.2

# (선택) 경고 줄이기: protobuf/pillow/requests/jedi 버전 맞춤
!pip install -q "protobuf<6,>=5.28.0" "pillow<12.0" "requests==2.32.4" "jedi>=0.16"

# 설치 확인(ml_dtypes는 0.5.1+면 정상)
!pip show ml-dtypes | sed -n '1,5p'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 

**실행 후**
> 세션 다시 시작(런타임 → 세션 다시 시작)

# 셀 B - 버전/GPU 확인
목적 : 설치 검증

In [1]:
# [한글] 설치된 버전 및 GPU 인식 확인
import numpy as np, tensorflow as tf
print("NumPy:", np.__version__)       # 기대: 2.0.x
print("TensorFlow:", tf.__version__)  # 기대: 2.20.0
print("GPU:", tf.test.gpu_device_name() or "CPU 사용")


NumPy: 2.0.2
TensorFlow: 2.20.0
GPU: /device:GPU:0


# 셀 C - 기본 임포트/로그 억제
목적 : 불필요한 로그 줄이고 기본 도구 준비

In [2]:
# [한글] 학습에 자주 쓰는 라이브러리 임포트 + 로그 억제
from __future__ import absolute_import, division, print_function, unicode_literals
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time, os
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   # TF Info/Warning 숨김
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"  # (Mac용 워크어라운드, 넣어도 무해)

# 셀 D - Google Drive 마운트
목적 : 드라이브 연결

In [3]:
# [한글] Google Drive 마운트 (권한 승인 필요)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**특이사항** : Opnebot에서 학습 시킨 데이터 ZIP 파일은
마운트가 끝나면, Drive의 루트에 OpenBotData 같은 새 폴더를 만들고, PC에서 ZIP 두 개를 직접 Drive로 업로드! (드라이브 웹 UI나 Colab 파일창 상단 업로드 버튼 사용)

# 셀 E - ZIP 해제(충돌 방지 분리 폴더)
목적 : ZIP을 /content/data/run1, run2에 해제

In [4]:
# [한글] 같은 이름 파일 덮어쓰기 방지용으로 각각 폴더에 풀기
!rm -rf /content/data && mkdir -p /content/data/run1 /content/data/run2
!unzip -oq "/content/drive/MyDrive/OpenBotData/20251031_145012.zip" -d /content/data/run1
!unzip -oq "/content/drive/MyDrive/OpenBotData/20251031_145241.zip" -d /content/data/run2
!ls -al /content/data

total 16
drwxr-xr-x 4 root root 4096 Oct 31 08:46 .
drwxr-xr-x 1 root root 4096 Oct 31 08:46 ..
drwxr-xr-x 4 root root 4096 Oct 31 08:46 run1
drwxr-xr-x 4 root root 4096 Oct 31 08:46 run2


# 셀 F - 저장소 받기 + 파이썬 경로 추가
목적 : OpenBot 코드를 가져오고, 파이썬이 찾을 수 있게 경로 추가

In [5]:
# [한글] OpenBot 저장소가 없으면 clone, 있으면 최신으로 갱신
import os, sys, importlib, subprocess, shlex

if not os.path.isdir("/content/OpenBot/.git"):
    !git clone https://github.com/Jinhan0603/OpenBot.git /content/OpenBot
else:
    # (선택) 최신 갱신
    !git -C /content/OpenBot pull --ff-only

# 파이썬 모듈 경로 추가
if "/content/OpenBot/policy" not in sys.path:
    sys.path.insert(0, "/content/OpenBot/policy")
importlib.invalidate_caches()

# openbot 패키지 폴더 존재 확인
assert os.path.isdir("/content/OpenBot/policy/openbot"), "openbot 패키지가 보이지 않습니다."
print("openbot path OK")


Cloning into '/content/OpenBot'...
remote: Enumerating objects: 15303, done.
remote: Counting objects: 100% (555/555), done.
remote: Compressing objects: 100% (362/362), done.
remote: Total 15303 (delta 281), reused 193 (delta 193), pack-reused 14748 (from 3)
Receiving objects: 100% (15303/15303), 235.27 MiB | 17.37 MiB/s, done.
Resolving deltas: 100% (8448/8448), done.
Updating files: 100% (2153/2153), done.
openbot path OK


# 셀 G - OpenBot 임포트 확인
목적 : 실제로 import가 되는지 확인

In [6]:
# [한글] OpenBot 모듈 임포트 테스트
from openbot import dataloader, data_augmentation, utils, train
print("openbot import OK")

Default GPU Device:/device:GPU:0
openbot import OK


# 셀 H - OpenBot이 기대하는 폴더 구조로 복사
목적 : 노트북/코드가 찾는 표준 경로에 데이터 배치

In [7]:
# [한글] OpenBot 표준 데이터 경로에 복사
!mkdir -p /content/OpenBot/policy/dataset/train_data/my_dataset_1
!mkdir -p /content/OpenBot/policy/dataset/test_data/my_dataset_2
!cp -r /content/data/run1 /content/OpenBot/policy/dataset/train_data/my_dataset_1/
!cp -r /content/data/run2 /content/OpenBot/policy/dataset/test_data/my_dataset_2/

# 확인
!find /content/OpenBot/policy/dataset -maxdepth 3 -type d -print


/content/OpenBot/policy/dataset
/content/OpenBot/policy/dataset/train_data
/content/OpenBot/policy/dataset/train_data/my_dataset_1
/content/OpenBot/policy/dataset/train_data/my_dataset_1/run1
/content/OpenBot/policy/dataset/test_data
/content/OpenBot/policy/dataset/test_data/my_dataset_2
/content/OpenBot/policy/dataset/test_data/my_dataset_2/run2
/content/OpenBot/policy/dataset/uploaded


# 셀 I - 데이터 경로 지정
> 노트북 기본 예시는 dataset_dir = "dataset"(상대경로)인데, Colab의 작업 디렉터리에 따라 빗나갈 수 있어요. openbot이 가지고 있는 절대경로를 쓰면 이후 단계와 정확히 일치합니다.



In [8]:
# [한글] 학습에 사용할 데이터셋 경로 지정
# - openbot 패키지의 기본 경로를 그대로 사용하면 이후 단계와 경로가 일치합니다.
import os
import openbot as ob

dataset_dir = ob.dataset_dir                     # /content/OpenBot/policy/dataset
dataset_name = "openbot"                         # 표기용 이름
train_data_dir = os.path.join(dataset_dir, "train_data")
test_data_dir  = os.path.join(dataset_dir, "test_data")

print("dataset_dir   :", dataset_dir)
print("train_data_dir:", train_data_dir)
print("test_data_dir :", test_data_dir)
assert os.path.isdir(train_data_dir), "train_data_dir 폴더가 없습니다."
assert os.path.isdir(test_data_dir),  "test_data_dir 폴더가 없습니다."


dataset_dir   : /content/OpenBot/policy/dataset
train_data_dir: /content/OpenBot/policy/dataset/train_data
test_data_dir : /content/OpenBot/policy/dataset/test_data


# 셀 J - 폴더 구조 점검 및 자동 정리(필요 시)


> ZIP을 run1, run2 같은 중간 폴더에 풀면 세션 폴더가 한 단계 더 깊어질 수 있어요. 이 셀은 그런 경우를 자동으로 올바른 깊이로 이동시켜 줍니다.



In [9]:
# [한글] 데이터 구조가 올바른지 검사하고, 한 단계 더 깊게 들어간 경우 자동으로 평탄화합니다.
# 기대 구조:
#  train_data/
#    └─ my_dataset_1/
#         └─ <각 세션>/ (images/, sensor_data/)
#  test_data/
#    └─ my_dataset_2/
#         └─ <각 세션>/ (images/, sensor_data/)
from pathlib import Path
import shutil

def ensure_flat(ds_root: Path):
  for ds in ds_root.iterdir():
    if not ds.is_dir():
      continue
    # 올바른 경우: ds/<session>/sensor_data 존재
    ok = list(ds.glob("*/sensor_data"))
    if ok:
      continue
    # 잘못된 경우(한 단계 더 깊음): ds/*/*/sensor_data -> 세션 폴더를 한 단계 위로 이동
    deep = list(ds.glob("*/*/sensor_data"))
    for sd in deep:
      session = sd.parent
      target_parent = ds
      target = target_parent / session.name
      if target.exists():
        i = 1
        while (target_parent / f"{session.name}_{i}").exists():
          i += 1
        target = target_parent / f"{session.name}_{i}"
      print(f"[move] {session} -> {target_parent}")
      shutil.move(str(session), str(target))

ensure_flat(Path(train_data_dir))
ensure_flat(Path(test_data_dir))

# 요약: 세션/이미지 개수
def count_sessions(root: str) -> int:
  return sum(1 for p in Path(root).glob("*/") if p.is_dir())
def count_imgs(root: str) -> int:
  return len(list(Path(root).glob("*/*/images/*")))

print("train sessions:", count_sessions(train_data_dir), "images:", count_imgs(train_data_dir))
print("test  sessions:", count_sessions(test_data_dir),  "images:", count_imgs(test_data_dir))


train sessions: 1 images: 1897
test  sessions: 1 images: 2003


# 셀 K - 샘플 경로 빠른 확인
> 세션/이미지 경로가 기대대로 보이면 다음 섹션(‘Hyperparameters’)로 넘어가면 됩니다.

In [10]:
# [한글] 샘플 이미지 경로를 몇 개 출력해 구조가 맞는지 빠르게 확인합니다.
from pathlib import Path

sample_train = list(Path(train_data_dir).glob("*/*/images/*"))[:3]
sample_test  = list(Path(test_data_dir).glob("*/*/images/*"))[:3]

print("sample train:", [str(p) for p in sample_train] or "없음")
print("sample test :", [str(p) for p in sample_test]  or "없음")


sample train: ['/content/OpenBot/policy/dataset/train_data/my_dataset_1/run1/images/7851_crop.jpeg', '/content/OpenBot/policy/dataset/train_data/my_dataset_1/run1/images/7840_crop.jpeg', '/content/OpenBot/policy/dataset/train_data/my_dataset_1/run1/images/8209_crop.jpeg']
sample test : ['/content/OpenBot/policy/dataset/test_data/my_dataset_2/run2/images/1799_crop.jpeg', '/content/OpenBot/policy/dataset/test_data/my_dataset_2/run2/images/506_crop.jpeg', '/content/OpenBot/policy/dataset/test_data/my_dataset_2/run2/images/2346_crop.jpeg']


# Hyperparameters
<a id='hyperparameters'></a>

You may have to tune the learning rate and batch size depending on your available compute resources and dataset. As a general rule of thumb, if you increase the batch size by a factor of n, you can increase the learning rate by a factor of sqrt(n). In order to accelerate training and make it more smooth, you should increase the batch size as much as possible. In our paper we used a batch size of 128. For debugging and hyperparamter tuning, you can set the number of epochs to a small value like 10. If you want to train a model which will achieve good performance, you should set it to 50 or more. In our paper we used 100.

In [ ]:
params = train.Hyperparameters()

params.MODEL = "pilot_net"  # choices: "pilot_net","cil_mobile","cil_mobile_fast","cil"
params.POLICY = "autopilot"  # choices: "autopilot","point_goal_nav"
params.TRAIN_BATCH_SIZE = 128
params.TEST_BATCH_SIZE = 16
params.LEARNING_RATE = 0.0003
params.NUM_EPOCHS = 100
params.BATCH_NORM = True  # use batch norm (recommended)
params.FLIP_AUG = False  # flip image and controls as augmentation (only autopilot)
params.CMD_AUG = False  # randomize high-level command as augmentation (only autopilot)
params.USE_LAST = False  # resume training from last checkpoint
params.WANDB = False
# policy = "autopilot": images are expected to be 256x96 - no cropping required
# policy = "point_goal_nav": images are expected to be 160x120 - cropping to 160x90
params.IS_CROP = params.POLICY == "point_goal_nav"

# Pre-process the dataset

In [ ]:
tr = train.Training(params)
tr.dataset_name = dataset_name
tr.train_data_dir = train_data_dir
tr.test_data_dir = test_data_dir

Running this for the first time will take some time. This code will match image frames to the controls (labels) and indicator signals (commands).  By default, data samples where the vehicle was stationary will be removed. If this is not desired, you need to set `tr.remove_zeros = False`. If you have made any changes to the sensor files, changed `remove_zeros` or moved your dataset to a new directory, you need to set `tr.redo_matching = True`.

In [ ]:
tr.redo_matching = False
tr.remove_zeros = True
train.process_data(tr)

In [ ]:
import threading


def broadcast(event, payload=None):
    print(event, payload)


event = threading.Event()
my_callback = train.MyCallback(broadcast, event)

In the next step, you can convert your dataset to a tfrecord, a data format optimized for training. You can skip this step if you already created a tfrecord before or if you want to train using the files directly.

In [ ]:
train.create_tfrecord(my_callback, policy=tr.hyperparameters.POLICY)

# Load the dataset

If you did not create a tfrecord and want to load and buffer files from disk directly, set `no_tf_record = True`.

In [ ]:
no_tf_record = False

In [ ]:
if no_tf_record:
    tr.train_data_dir = train_data_dir
    tr.test_data_dir = test_data_dir
    train.load_data(tr, verbose=0)
else:
    tr.train_data_dir = os.path.join(dataset_dir, "tfrecords/train.tfrec")
    tr.test_data_dir = os.path.join(dataset_dir, "tfrecords/test.tfrec")
    train.load_tfrecord(tr, verbose=0)

In [ ]:
# Select interactive backend to show inline
%matplotlib inline
utils.show_batch(dataset=tr.train_ds, policy=tr.hyperparameters.POLICY, model=None)

# Training

The number of epochs is proportional to the training time. One epoch means going through the complete dataset once. Increasing `NUM_EPOCHS` will mean longer training time, but generally leads to better performance. To get familiar with the code it can be set to small values like `5` or `10`. To train a model that performs well, it should be set to values between `50` and `200`. Setting `USE_LAST` to `true` will resume the training from the last checkpoint. The default values are `NUM_EPOCHS = 100` and `USE_LAST = False`. They are set in [Hyperparameters](#hyperparameters).

In [ ]:
# Create the directory if it doesn't exist
os.makedirs(os.path.join(os.getcwd(), "models", tr.model_name), exist_ok=True)

# params.NUM_EPOCHS = 200
# params.USE_LAST = True
train.do_training(tr, my_callback, verbose=1)

# Evaluation

The loss and mean absolute error should decrease. This indicates that the model is fitting the data well. The custom metrics (direction and angle) should go towards 1. These provide some additional insight to the training progress. The direction metric measures weather or not predictions are in the same direction as the labels. Similarly the angle metric measures if the prediction is within a small angle of the labels. The intuition is that driving in the right direction with the correct steering angle is most critical part for good final performance.

### Plot metrics

In [ ]:
x = np.arange(tr.INITIAL_EPOCH + 1, tr.history.params["epochs"] + 1, 1)

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["loss"], label="loss")
plt.plot(x, tr.history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.savefig(os.path.join(tr.log_path, "loss.png"), bbox_inches="tight")

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["mean_absolute_error"], label="mean_absolute_error")
plt.plot(
    x, tr.history.history["val_mean_absolute_error"], label="val_mean_absolute_error"
)
plt.xlabel("Epoch")
plt.ylabel("Mean Absolute Error")
plt.legend(loc="upper right")
plt.savefig(os.path.join(tr.log_path, "error.png"), bbox_inches="tight")

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["direction_metric"], label="direction_metric")
plt.plot(x, tr.history.history["val_direction_metric"], label="val_direction_metric")
plt.xlabel("Epoch")
plt.ylabel("Direction Metric")
plt.legend(loc="lower right")
plt.savefig(os.path.join(tr.log_path, "direction.png"), bbox_inches="tight")

In [ ]:
plt.figure().gca().xaxis.get_major_locator().set_params(integer=True)
plt.plot(x, tr.history.history["angle_metric"], label="angle_metric")
plt.plot(x, tr.history.history["val_angle_metric"], label="val_angle_metric")
plt.xlabel("Epoch")
plt.ylabel("Angle Metric")
plt.legend(loc="lower right")
plt.savefig(os.path.join(tr.log_path, "angle.png"), bbox_inches="tight")

### Save tf lite models for best train, best val and last checkpoint

In [ ]:
best_train_checkpoint = "cp-best-train.ckpt"
best_train_tflite = utils.generate_tflite(tr.checkpoint_path, best_train_checkpoint)
utils.save_tflite(best_train_tflite, tr.checkpoint_path, "best-train")
best_train_index = np.argmin(np.array(tr.history.history["loss"]))
print(
    "Best Train Checkpoint (epoch %s) - angle: %.4f, val_angle: %.4f, direction: %.4f, val_direction: %.4f"
    % (
        best_train_index,
        tr.history.history["angle_metric"][best_train_index],
        tr.history.history["val_angle_metric"][best_train_index],
        tr.history.history["direction_metric"][best_train_index],
        tr.history.history["val_direction_metric"][best_train_index],
    )
)

In [ ]:
best_val_checkpoint = "cp-best-val.ckpt"
best_val_tflite = utils.generate_tflite(tr.checkpoint_path, best_val_checkpoint)
utils.save_tflite(best_val_tflite, tr.checkpoint_path, "best")
utils.save_tflite(best_val_tflite, tr.checkpoint_path, "best-val")
best_val_index = np.argmin(np.array(tr.history.history["val_loss"]))
print(
    "Best Val Checkpoint (epoch %s) - angle: %.4f, val_angle: %.4f, direction: %.4f, val_direction: %.4f"
    % (
        best_val_index,
        tr.history.history["angle_metric"][best_val_index],
        tr.history.history["val_angle_metric"][best_val_index],
        tr.history.history["direction_metric"][best_val_index],
        tr.history.history["val_direction_metric"][best_val_index],
    )
)

In [ ]:
last_checkpoint = "cp-last.ckpt"
last_tflite = utils.generate_tflite(tr.checkpoint_path, last_checkpoint)
utils.save_tflite(last_tflite, tr.checkpoint_path, "last")
print(
    "Last Checkpoint - angle: %.4f, val_angle: %.4f, direction: %.4f, val_direction: %.4f"
    % (
        tr.history.history["angle_metric"][-1],
        tr.history.history["val_angle_metric"][-1],
        tr.history.history["direction_metric"][-1],
        tr.history.history["val_direction_metric"][-1],
    )
)

### Evaluate the best model (train loss) on the training set

In [ ]:
best_train_model = utils.load_model(
    os.path.join(tr.checkpoint_path, best_train_checkpoint),
    tr.loss_fn,
    tr.metric_list,
    tr.custom_objects,
)
loss, mae, direction, angle = best_train_model.evaluate(
    tr.train_ds,
    steps=tr.image_count_train / tr.hyperparameters.TRAIN_BATCH_SIZE,
    verbose=1,
)

In [ ]:
utils.show_batch(
    dataset=tr.train_ds, policy=tr.hyperparameters.POLICY, model=best_train_model
)

In [ ]:
utils.compare_tf_tflite(best_train_model, best_train_tflite)

### Evaluate the best model (val loss) on the validation set

In [ ]:
best_val_model = utils.load_model(
    os.path.join(tr.checkpoint_path, best_val_checkpoint),
    tr.loss_fn,
    tr.metric_list,
    tr.custom_objects,
)
loss, mae, direction, angle = best_val_model.evaluate(
    tr.test_ds,
    steps=tr.image_count_test / tr.hyperparameters.TEST_BATCH_SIZE,
    verbose=1,
)

In [ ]:
utils.show_batch(
    dataset=tr.test_ds, policy=tr.hyperparameters.POLICY, model=best_val_model
)

In [ ]:
utils.compare_tf_tflite(best_val_model, best_val_tflite)

# Save the notebook as HTML

In [ ]:
utils.save_notebook()
current_file = "policy_learning.ipynb"
output_file = os.path.join(tr.log_path, "notebook.html")
utils.output_HTML(current_file, output_file)